# Synthesize a Set of PPI

In [ ]:
import os
import time
import glob
import numpy as np
import datetime

import chart

from netCDF4 import Dataset

In [ ]:
def makevalues(mx=-6, my=9, sx=2, sy=6, rho=0.7):
    v = np.zeros((360, 1000), dtype=float)
    r = 1.0e-3 * np.arange(v.shape[1]) * 60.0
    a = np.deg2rad(np.arange(v.shape[0]))
    x = np.outer(np.sin(a), r)
    y = np.outer(np.cos(a), r)

    x += 2.5 * np.random.rand(v.shape[0], v.shape[1])
    y += 2.5 * np.random.rand(v.shape[0], v.shape[1])

    v = np.exp(- 1 / (2 * (1 - rho ** 2)) * (((x - mx) / sx) ** 2
                                           + ((y - my) / sy) ** 2
                 + 2 * rho * ((x - mx) / sx) * ((y - my) / sy)))
    v = 53 + 10 * np.log10(v + 1.0e-7)
    v[v < -10] = np.nan
    return v

In [ ]:
def writefile(values, a, r, t, e=2.4, prefix='DX', verbose=0):
    timestr = time.strftime('%Y%m%d-%H%M%S', time.localtime(t))
    filename = f'data/{prefix}-{timestr}-E{e:.1f}-Z.nc'
    if verbose:
        print(f'{filename}')
    ncid = Dataset(filename, mode='w', format='NETCDF4_CLASSIC')
    dim_a = ncid.createDimension('Azimuth', len(a))
    dim_r = ncid.createDimension('Gate', len(r))
    ncid.setncatts({
        'TypeName': 'Corrected_Intensity',
        'DataType': 'RadialSet',
        'ScanType': 'PPI',
        'Latitude': 35.23682,
        'Longitude': -97.46381,    
        'Height': np.float32(0.0), 
        'Time': t,
        'FractionalTime': np.float32(0.0),
        'attributes': 'Nyquist_Vel Unit radarName vcp ColorMap',
        'Nyquist_Vel-unit': 'MetersPerSecond',
        'Nyquist_Vel-value': np.float32(15.70681),
        'Unit-unit': 'dimensionless',
        'Unit-value': 'dBZ',
        'radarName-unit': 'dimensionless',
        'radarName-value': 'PX-1000',
        'vcp-unit': 'dimensionless',
        'vcp-value': '1',
        'ColorMap-unit': 'dimensionless',
        'ColorMap-value': 'Reflectivity',
        'Elevation': np.float32(e),
        'ElevationUnits': 'Degrees',
        'Azimuth': np.float32(0),
        'AzimuthUnits': 'Degrees',
        'RangeToFirstGate': np.float32(0),
        'RangeToFirstGateUnits': 'Meters',
        'MissingData': np.float32(-99900.0),
        'RangeFolded': np.float32(-99901.0),
        'RadarParameters': 'PRF PulseWidth MaximumRange',
        'PRF-unit': 'Hertz',
        'PRF-value': int(2000),
        'PulseWidth-unit': 'MicroSeconds',
        'PulseWidth-value': np.float32(69),
        'MaximumRange-unit': 'KiloMeters',
        'MaximumRange-value': np.float32(r[-1]),
        'ProcessParameters': 'Noise Calib Censor',
        'NoiseH-unit': 'dB-ADU',
        'NoiseH-value': np.float32(13.5),
        'NoiseV-unit': 'dB-ADU',
        'NoiseV-value': np.float32(9.5),
        'CalibH-unit': 'dB',
        'CalibH-value': np.float32(12.),
        'CalibV-unit': 'dB',
        'CalibV-value': np.float32(15.5),
        'CalibD1-unit': 'dB',
        'CalibD1-value': np.float32(0),
        'CalibD2-unit': 'dB',
        'CalibD2-value': np.float32(0),
        'CalibP1-unit': 'Degrees',
        'CalibP1-value': np.float32(0),
        'CalibP2-unit': 'Degrees',
        'CalibP2-value': np.float32(0),
        'CensorThreshold-unit': 'dB',
        'CensorThreshold-value': np.float32(-3.0),
        'CompressionMethod': 'Static',
        'MomentMethod': 'Synth',
        'Waveform': 'none',
        'CreatedBy': 'Python',
        'ContactInformation': 'http://arrc.ou.edu'
    })
    az = ncid.createVariable('Azimuth', np.float32, ('Azimuth',))
    az.Units = 'Degrees'
    el = ncid.createVariable('Elevation', np.float32, ('Azimuth',))
    el.Units = 'Degrees'
    bw = ncid.createVariable('Beamwidth', np.float32, ('Azimuth',))
    bw.Units = 'Degrees'
    gw = ncid.createVariable('GateWidth', np.float32, ('Azimuth',))
    gw.Units = 'Meters'
    v = ncid.createVariable('Corrected_Intensity', np.float32, ('Azimuth', 'Gate'))
    v.Units = 'dBZ'
    az[:] = a
    el[:] = e
    bw[:] = 1.0
    gw[:] = 60.0
    v[:, :] = values
    ncid.close()

In [ ]:
if not os.path.exists('data'):
    os.mkdir('data')

t = int(time.mktime(time.strptime('2023-07-04 11:20:34', '%Y-%m-%d %H:%M:%S')))
r = 1.0e-3 * np.arange(1000) * 60.0
a = np.arange(360)

for mx in np.arange(20, -20.5, -0.25):
    t += 120
    my = 0.2 * mx
    print(f'mx = {mx:5.1f}   my = {my:5.1f}   ', end='')
    values = makevalues(mx=mx, my=my)
    writefile(values, a=a, r=r, t=t, verbose=1)

In [ ]:
files = sorted(glob.glob('data/*.nc'))
file = files[0]

with Dataset(file, mode='r') as nc:
    name = nc.getncattr('TypeName')
    elev = np.array(nc.variables['Elevation'][:], dtype=np.float32)
    azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
    gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
    values = np.array(nc.variables[name][:], dtype=np.float32)
    values[values < -90] = np.nan
    longitude = nc.getncattr('Longitude')
    latitude = nc.getncattr('Latitude')
    sweepElev = nc.getncattr('Elevation')
    sweepTime = nc.getncattr('Time')
    symbol = file.split('.')[-2].split('-')[-1]

In [ ]:
overlay = chart.atlas.Overlay((longitude, latitude))

radii = np.concatenate(([1.0], np.arange(10.0, 31.0, 10.0)))
overlay.setRingRadii(radii)
overlay.load()

r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
a = np.deg2rad(azim)
timestr = datetime.datetime.utcfromtimestamp(sweepTime).strftime('%Y/%m/%d %H:%M:%S')
title = f'{timestr} UTC  EL: {sweepElev:.2f}°'
symbol = 'Z'

ppi = chart.chart.Image(a, r, values, style=symbol, overlay=overlay, title=title, figsize=(800, 600), maxrange=25.0)

In [ ]:
def populateImage(file):
    with Dataset(file, mode='r') as nc:
        name = nc.getncattr('TypeName')
        azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
        gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
        values = np.array(nc.variables[name][:], dtype=np.float32)
        values[values < -90] = np.nan
        sweepTime = nc.getncattr('Time')
        timestr = datetime.datetime.utcfromtimestamp(sweepTime).strftime('%Y/%m/%d %H:%M:%S')
        title = f'{timestr} UTC  EL: {sweepElev:.2f}°'
        r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
        a = np.deg2rad(azim)
        symbol = file.split('.')[-2].split('-')[-1]
    ppi.set_data(values, style=symbol, a=a, r=r, title=title)

In [ ]:
populateImage(files[0])
ppi.image()

In [ ]:
populateImage(files[24])
ppi.image()

In [ ]:
populateImage(files[-1])
ppi.image()